# MOwNiT laboratorium nr 1 - Arytmetyka komputerowa

### Zadanie 1
Wykorzystaj funkcję finfo z biblioteki numpy aby określić precyzję i zakres różnych typów danych reprezentujących liczby zmienno-przecinkowe. Wyniki badania przedstaw w postaci zestawienia tabelrycznego. 

In [0]:
!pip install bitstring
!pip install tabulate
import numpy as np
from tabulate import tabulate
import bitstring as bs


def get_range(x):
    return " {} - {}".format(np.finfo(x).min, np.finfo(x).max)


def get_eps(x):
    return np.finfo(x).eps


table = [['float16', get_range(np.float16), get_eps(np.float16)],
         ['float32', get_range(np.float32), get_eps(np.float32)],
         ['float64', get_range(np.float64), get_eps(np.float64)],
         ['float128', get_range(np.float128), get_eps(np.float128)]]

print(tabulate(table, headers=['Variable', 'Range', 'Eps'], tablefmt="fancy_grid", floatfmt=".25f"))



╒════════════╤════════════════════════════════════════════════════╤═════════════════════════════╕
│ Variable   │ Range                                              │                         Eps │
╞════════════╪════════════════════════════════════════════════════╪═════════════════════════════╡
│ float16    │ -65504.0 - 65504.0                                 │ 0.0009765625000000000000000 │
├────────────┼────────────────────────────────────────────────────┼─────────────────────────────┤
│ float32    │ -3.4028234663852886e+38 - 3.4028234663852886e+38   │ 0.0000001192092895507812500 │
├────────────┼────────────────────────────────────────────────────┼─────────────────────────────┤
│ float64    │ -1.7976931348623157e+308 - 1.7976931348623157e+308 │ 0.0000000000000002220446049 │
├────────────┼────────────────────────────────────────────────────┼─────────────────────────────┤
│ float128   │ -inf - inf                                         │ 0.0000000000000000001084202 │
╘════════════╧══════

### Zadanie 2 
Napisz funkcje, która dla dowolnej liczby zmiennoprzecinkowej w precyzji 64-bitowej (w standardzie IEEE 754) przedstawi w wersji dziesiętnej i binarnej składniki liczby: znak, wykładnik i mantysa. Można wykorzystać między innymi bibliotekę bitstring. Zbadaj jak reprezentowaną są poszczególne składniki tego formatu: zakres, sposób kodowania liczby etc. 

In [0]:
def convert_to_iee754(x):
    y = bs.BitArray(float=x, length=64).bin
    return y[0], y[1:12], y[12:64], x


print(convert_to_iee754(1.2))
print(convert_to_iee754(1.75))
print(convert_to_iee754(1.25))



('0', '01111111111', '0011001100110011001100110011001100110011001100110011', 1.2)
('0', '01111111111', '1100000000000000000000000000000000000000000000000000', 1.75)
('0', '01111111111', '0100000000000000000000000000000000000000000000000000', 1.25)


Liczby zmiennoprzecinkowej w precyzji 64-bitowej (w standardzie IEEE 754), są zapisywane w komputerze w następujacy sposób : 
  - znak (1 bit)
  - cecha (11 bitów)
  - mantysa (52 bity)
![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/IEEE_754_Double_Floating_Point_Format.svg/1920px-IEEE_754_Double_Floating_Point_Format.svg.png)

Zamiana liczby z postaci binarnej na wartości dziesiętną odbywa się w następujący sposób :           
![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/61345d47f069d645947b9c0ab676c75551f1b188)  
lub też inny zapis :       
![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/5f677b27f52fcd521355049a560d53b5c01800e1)

Najmniejszą liczbe jako da się zapisać za pomocą tego zapisu to :  
0 00000000001 0000000000000000000000000000000000000000000000000000 ≙ +2^(−1022) × 1           
≈ 2.2250738585072014 × 10^(−308)

Zatomiast najwieksza liczba zapisana w tym systemie będzie miala postać : 
0 11111111110 1111111111111111111111111111111111111111111111111111 ≙  +2^1023 × (1 + (1 − 2^(−52)))  
≈ 1.7976931348623157 × 10^(308)

### Zadanie 3 
Wykorzystaj funkcję nextafter() z biblioteki numpy (lub inny sposób) i sprawdź rozdzielczość typu float (jakie mają wartości najbliższe sąsiadujące liczby float). Zaprezentuj wynik korzystając z funkcji z Zadania 2. 

In [0]:
def eps(x):
    print("Value: {}, the nearest value: {}".format(x, np.nextafter(x, 1)))
    print(convert_to_iee754(x))
    print(convert_to_iee754(np.nextafter(x, 1)))


print(eps(1.2))    
print(eps(1.25)) 
print(eps(1.7)) 

Value: 1.2, the nearest value: 1.1999999999999997
('0', '01111111111', '0011001100110011001100110011001100110011001100110011', 1.2)
('0', '01111111111', '0011001100110011001100110011001100110011001100110010', 1.1999999999999997)
None
Value: 1.25, the nearest value: 1.2499999999999998
('0', '01111111111', '0100000000000000000000000000000000000000000000000000', 1.25)
('0', '01111111111', '0011111111111111111111111111111111111111111111111111', 1.2499999999999998)
None
Value: 1.7, the nearest value: 1.6999999999999997
('0', '01111111111', '1011001100110011001100110011001100110011001100110011', 1.7)
('0', '01111111111', '1011001100110011001100110011001100110011001100110010', 1.6999999999999997)
None


### Zadanie 5
Sprawdź czym jest postać znormalizowana liczby typu float. Poniższy program generuje ciąg liczb. Wyświetl ich postać binarną i sprawdź czy są w postaci znormalizowanej (napisz stosowny kod, który to sprawdza). 

Znormalizowana postać liczby zmiennoprzecinkowej to taka, w której
mantysa spełnia nierówność :  
p > | m | ≥ 1   
, gdzie p – podstawy systemu   
Według tej definicji postacią znormalizowaną dla zapisów:  
325 x 10^20 = 32,5 x 10^21 = 3,25 x 10^22 = 0,0325 x 10^24  
jest jedynie zapis 3,25 x 10^22.

In [0]:
def is_normalize(x):
    return 'Yes' if x[1] == '00000000000' else 'No'


output = []
a = np.float32(1.1)
for i in range(0, 160):
    a = a / np.float32(2.0)
    sign, exponent, significand, value = convert_to_iee754(a)
    output.append([is_normalize(convert_to_iee754(a)), value, sign, exponent, significand ])

print(tabulate(output, headers=['Is denormalize?', 'Value', 'Sign', 'Exponent', 'Significand'], tablefmt="fancy_grid"))

╒═══════════════════╤═════════════╤════════╤═════════════╤══════════════════════════════════════════════════════╕
│ Is denormalize?   │       Value │   Sign │    Exponent │                                          Significand │
╞═══════════════════╪═════════════╪════════╪═════════════╪══════════════════════════════════════════════════════╡
│ No                │ 0.55        │      0 │ 01111111110 │ 0001100110011001100110100000000000000000000000000000 │
├───────────────────┼─────────────┼────────┼─────────────┼──────────────────────────────────────────────────────┤
│ No                │ 0.275       │      0 │ 01111111101 │ 0001100110011001100110100000000000000000000000000000 │
├───────────────────┼─────────────┼────────┼─────────────┼──────────────────────────────────────────────────────┤
│ No                │ 0.1375      │      0 │ 01111111100 │ 0001100110011001100110100000000000000000000000000000 │
├───────────────────┼─────────────┼────────┼─────────────┼──────────────────────────────

### Zadanie 4
Porównać reprezentację bitową liczby 1/3 dla typów float o różnej precyzji: float16, float32, float64. Sprawdź co się dzieje gdy zmienna o mniejszej precyzji konwertowana jest do wyższej. 

In [0]:
fl_16 = np.float16(1/3)
fl_32 = np.float32(1/3)
fl_64 = np.float64(1/3)
fl_128 = np.float128(1/3)


table = [['float_16', convert_to_iee754(fl_16)],
         ['float_32', convert_to_iee754(fl_32)],
         ['float_64', convert_to_iee754(fl_64)],
         ['convert float_16 to float_32', convert_to_iee754(fl_16.astype(np.float32))],
         ['convert float_16 to float_64', convert_to_iee754(fl_16.astype(np.float64))]]
print(tabulate(table, headers=['L.P.', 'Value'], tablefmt="fancy_grid", floatfmt=".10f"))

╒══════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════╕
│ L.P.                         │ Value                                                                                            │
╞══════════════════════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════╡
│ float_16                     │ ('0', '01111111101', '0101010101000000000000000000000000000000000000000000', 0.3333)             │
├──────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ float_32                     │ ('0', '01111111101', '0101010101010101010101100000000000000000000000000000', 0.33333334)         │
├──────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ float_64                     │ ('0', '01111111101', '010101010101010101010

Z powyższej tabelki wynika, że im większą precyzje stosujemy tym więcej wykorzystujemy bitów do zapisu liczby tym samym liczba jest zapisana w dokładniejszy sposób.
 

### Zadanie 6
Wyjaśnij dlaczego poniższy program nie działa prawidłowo. Sprawdź co się stanie gdy użyjemy typów o mniejszej precyzji. Zaproponuj inny sposób sumowania, który będzie dawał dokładne wyniki. 

In [0]:
N = 100000000
C = 0.09531258654533566;

sum = 0.0 
for i in range(0, N):
    sum += C
    
print(sum)
print(N*C)

9531258.666974738
9531258.654533565


Powyższy kod nie działa poprawnie, ponieważ sumując N-krotnie liczbę 0.09531258654533566 powstaje błąd, ten błąd wynika z dodawania ciągu liczb zmiennopozycyjnych o skończonej precyzji (zapis liczby 0.09531258654533566 w komputerze posiada błąd w zależności od użytej precyzji).
Aby zminimalizować ten błąd użyłem powszechnie znanego algorytmu kahana, który w każdej iteracji od sumy odejmuje błąd zapisu (eps).


In [0]:
import numpy as np
from tabulate import tabulate


def kahan_algorithm(x, n):
    sum = 0.0
    eps = 0.0
    for i in range(0, n):
        x_without_eps = x - eps
        tmp_sum = sum + x_without_eps
        eps = (tmp_sum - sum) - x_without_eps
        sum = tmp_sum
    return sum


def simple_resolver(x, n, sum):
    for i in range(0, n):
        sum += x
    return sum


N = 100000000
C = 0.09531258654533566
table = [['float_16', simple_resolver(np.float16(0.09531258654533566), 100000000, np.float16(0))],
         ['float_32', simple_resolver(np.float32(0.09531258654533566), 100000000, np.float32(0))],
         ['float_64', simple_resolver(np.float64(0.09531258654533566), 100000000, np.float64(0))],
         ['float_128', simple_resolver(np.float128(0.09531258654533566), 100000000, np.float128(0))],
         ['kahan_algorithm', kahan_algorithm(C, N)],
         ['correct_value', C*N]]

print(tabulate(table, headers=['L.P.', 'Value'], tablefmt="fancy_grid", floatfmt=".10f"))


╒═════════════════╤════════════════════╕
│ L.P.            │              Value │
╞═════════════════╪════════════════════╡
│ float_16        │     256.0000000000 │
├─────────────────┼────────────────────┤
│ float_32        │ 2097152.0000000000 │
├─────────────────┼────────────────────┤
│ float_64        │ 9531258.6669747382 │
├─────────────────┼────────────────────┤
│ float_128       │ 9531258.6545333061 │
├─────────────────┼────────────────────┤
│ kahan_algorithm │ 9531258.6545335650 │
├─────────────────┼────────────────────┤
│ correct_value   │ 9531258.6545335650 │
╘═════════════════╧════════════════════╛
